### Imports

In [3]:
import numpy as np
import random
# import to do training
from tpg.trainer import Trainer
# import to run an agent (always needed)
from tpg.agent import Agent
# faster training
import multiprocessing as mp
# visual tools
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt

In [4]:
def animate_path(sequence):
    current_map = np.zeros((10, 10))
    # add barrier
    for i in range(5):
        current_map[(4, i)] = 5
    # animate the run!
    for i in range(len(sequence)):
        time.sleep(0.5)
        if i == 0:
            current_map[sequence[i]] = 1
            clear_output(wait=True)
            print(0)
            print(current_map)
        else:
            current_map[sequence[i-1]] = 0
            current_map[sequence[i]] = 1
            clear_output(wait=True)
            print(i)
            print(current_map)

In [5]:
# example of above 
seq = [(0,1), (0,2), (0, 3), (1, 3), (2, 4)]
animate_path(seq)

4
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 5. 5. 5. 5. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [6]:
# todo: make generalizable for any GridWorld
def heat_map(sequence):
    x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    y = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    plt.xticks(ticks=np.arange(len(x)),labels=x)
    plt.yticks(ticks=np.arange(len(y)),labels=y)
    plt.imshow(sequence, cmap='cool')

### Grid World

In [7]:
# based (for now) on Downing's Figure 4.1
class GridWorldv1:
    def __init__(self, rows, cols, win_state, starting_state):  
        self.rows = rows
        self.cols = cols
        self.starting_state = starting_state
        self.win_state = win_state
        self.current_state = self.starting_state

    def sample_action (self):
        rand = random.uniform(0, 1)
        if (rand >= 0) and (rand < 0.25):
            return 0
        elif (rand >= 0.25) and (rand < 0.5):
            return 1
        elif (rand >= 0.5) and (rand < 0.75):
            return 2
        else:
            return 3
        
    def reset (self):
        self.current_state = self.starting_state
        return self.current_state
        
    def check_win (self):
        if self.current_state == self.win_state:
            return True
        return False

    # just reset for now...
    def close (self):
        self.current_state = self.starting_state
        return 1
    
    def step (self, action):
        # north
        if action == 0:
            next = (self.current_state[0] - 1, self.current_state[1])
        # south
        elif action == 1:
            next = (self.current_state[0] + 1, self.current_state[1])
        # east
        elif action == 2:
            next = (self.current_state[0], self.current_state[1] + 1)
        # west
        else:
            next = (self.current_state[0], self.current_state[1] - 1)

        terminate = False
        reward = 0
        # check if move is legal
        if (next[0] >= 0 and next[0] <= 9) and (next[1] >= 0 and next[1] <= 9):            
            illegal = 0
            for i in range(5):
                if next == (4, i):
                    illegal = 1
                    
            if (illegal == 0):
                self.current_state = next
                reward -= 0.01
                #print(self.current_state)
            else:
                #print('illegal move!')
                reward -= 1
                #terminate = True
        else:
            #print('out of bounds!')
            reward -= 1
            #terminate = True
        
        if self.check_win():
            reward += 100
            terminate = True
        
        return self.current_state, reward, terminate

### GridWorld Test

In [8]:
env = GridWorldv1(10, 10, (9, 0), (0, 0))
state = env.reset()
#action = agent.act(state)
action = env.sample_action()
state, reward, isDone = env.step(action)
print(state)
print(reward)
print(isDone)

(0, 0)
-1
False


### TPG Agent

In [11]:
def runAgent(args):
    agent = args[0]
    envName = args[1]
    scoreList = args[2]
    numEpisodes = args[3] # number of times to repeat game
    numFrames = args[4] 
    gen = args[5]
    
    # skip if task already done by agent
    if agent.taskDone(envName):
        print('Agent #' + str(agent.agentNum) + ' can skip.')
        scoreList.append((agent.team.id, agent.team.outcomes))
        return
    
    env = GridWorldv1(10, 10, (9, 0), (0, 0))
    valActs = 4
    
    limit = 100000000
    
    # create environment object from above
    scoreTotal = 0
    for ep in range(numEpisodes):
        state = env.reset()
        scoreEp = 0
        states = []
        step_map = np.zeros((10, 10))
        if numEpisodes > 1:
            numRandFrames = random.randint(0,30)
            limitReached = 0
            for i in range(numFrames): # frame loop
                if i < numRandFrames:
                    env.step(env.sample_action())
                    continue
                    
                # action selection
                act = agent.act(state)
                
                state, reward, isDone = env.step(act)
                # for heatmap
                step_map[state] = step_map[state] + 1
                states.append(state)
                #print('State: ' + str(state))
                scoreEp += reward
                       
                if i > limit:
                    print('Limit Reached!')
                    limitReached = True
                
                # animate the winners to see what they are doing
                if isDone:
                    print('win!')
                    animate_path(states)
                
                if isDone or limitReached:
                    break
                    
            print('Agent #' + str(agent.agentNum) + 
              ' | Ep #' + str(ep) + ' | Score: ' + str(scoreEp))
            
        scoreTotal += scoreEp
    scoreTotal /= numEpisodes
    env.close()
    agent.reward(scoreTotal, envName)
    scoreList.append((agent.team.id, agent.team.outcomes))

### All Together Now!

In [12]:
acts = 4
# pAtomic equal to 1 so teams are single nodes
    # starting off, teams don't reference teams
trainer = Trainer(actions=acts, teamPopSize=10, pActAtom=1.0)

envName = 'GridWorldv1'
allScores = []
for gen in range(100):
    scoreList = []

    agents = trainer.getAgents()

    for agent in agents:
        runAgent([agent, envName, scoreList, 10, 1000, gen])

    # apply scores, must do this when multiprocessing
    # because agents can't refer to trainer
    teams = trainer.applyScores(scoreList)

    trainer.evolve(tasks=[envName]) # go into next gen

    # an easier way to track stats than the above example
    scoreStats = trainer.fitnessStats

    allScores.append((scoreStats['min'], scoreStats['max'], scoreStats['average']))

    print('Gen: ' + str(gen))
    print('Results so far: ' + str(allScores))

print('Results:\nMin, Max, Avg')
for score in allScores:
    print(score[0],score[1],score[2])

31
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [5. 5. 5. 5. 5. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Agent #5 | Ep #5 | Score: 99.67999999999999
Agent #5 | Ep #6 | Score: -83.03000000000058
Agent #5 | Ep #7 | Score: -83.0500000000006
Agent #5 | Ep #8 | Score: -82.98000000000056
Agent #5 | Ep #9 | Score: -85.22000000000068
Agent #6 | Ep #0 | Score: -85.12000000000064
Agent #6 | Ep #1 | Score: -82.96000000000056
Agent #6 | Ep #2 | Score: -77.10000000000026
Agent #6 | Ep #3 | Score: -86.09000000000076
Agent #6 | Ep #4 | Score: -85.21000000000068
Agent #6 | Ep #5 | Score: -85.22000000000068
Agent #6 | Ep #6 | Score: -82.05000000000058
Agent #6 | Ep #7 | Score: -83.18000000000065
Agent #6 | Ep #8 | Score: -85.03000000000064
Agent #6 | Ep #9 | Score: -82.07000000

Agent #5 | Ep #6 | Score: -84.22000000000068
Agent #5 | Ep #7 | Score: -81.98000000000056
Agent #5 | Ep #8 | Score: -82.1300000000006
Agent #5 | Ep #9 | Score: -84.0500000000006
Agent #6 | Ep #0 | Score: -82.11000000000061
Agent #6 | Ep #1 | Score: -85.06000000000067
Agent #6 | Ep #2 | Score: -84.94000000000065
Agent #6 | Ep #3 | Score: -83.16000000000062
Agent #6 | Ep #4 | Score: -83.16000000000064
Agent #6 | Ep #5 | Score: -82.1000000000006
Agent #6 | Ep #6 | Score: -79.08000000000035
Agent #6 | Ep #7 | Score: -84.13000000000063
Agent #6 | Ep #8 | Score: -84.05000000000062
Agent #6 | Ep #9 | Score: -82.0700000000006
Agent #7 | Ep #0 | Score: -971.01
Agent #7 | Ep #1 | Score: -995
Agent #7 | Ep #2 | Score: -997
Agent #7 | Ep #3 | Score: -974.02
Agent #7 | Ep #4 | Score: -975.02
Agent #7 | Ep #5 | Score: -986.01
Agent #7 | Ep #6 | Score: -990
Agent #7 | Ep #7 | Score: -986.03
Agent #7 | Ep #8 | Score: -984.03
Agent #7 | Ep #9 | Score: -976
Agent #8 | Ep #0 | Score: -85.16000000000066
A

Agent #7 | Ep #5 | Score: -85.10000000000068
Agent #7 | Ep #6 | Score: -83.01000000000056
Agent #7 | Ep #7 | Score: -84.15000000000063
Agent #7 | Ep #8 | Score: -84.04000000000059
Agent #7 | Ep #9 | Score: -84.22000000000068
Agent #8 | Ep #0 | Score: -77.13000000000024
Agent #8 | Ep #1 | Score: -83.23000000000067
Agent #8 | Ep #2 | Score: -85.05000000000067
Agent #8 | Ep #3 | Score: -84.06000000000064
Agent #8 | Ep #4 | Score: -86.03000000000073
Agent #8 | Ep #5 | Score: -84.25000000000068
Agent #8 | Ep #6 | Score: -84.17000000000066
Agent #8 | Ep #7 | Score: -79.00000000000036
Agent #8 | Ep #8 | Score: -83.10000000000062
Agent #8 | Ep #9 | Score: -84.11000000000062
Agent #9 | Ep #0 | Score: -999
Agent #9 | Ep #1 | Score: -991.02
Agent #9 | Ep #2 | Score: -997.01
Agent #9 | Ep #3 | Score: -968.02
Agent #9 | Ep #4 | Score: -980
Agent #9 | Ep #5 | Score: -974.02
Agent #9 | Ep #6 | Score: -974.03
Agent #9 | Ep #7 | Score: -993
Agent #9 | Ep #8 | Score: -981.01
Agent #9 | Ep #9 | Score: -9

Agent #8 | Ep #4 | Score: -972.03
Agent #8 | Ep #5 | Score: -1000
Agent #8 | Ep #6 | Score: -978.03
Agent #8 | Ep #7 | Score: -977
Agent #8 | Ep #8 | Score: -992.01
Agent #8 | Ep #9 | Score: -973.03
Agent #9 | Ep #0 | Score: -955.1700000000001
Agent #9 | Ep #1 | Score: -973.14
Agent #9 | Ep #2 | Score: -980.16
Agent #9 | Ep #3 | Score: -978.1700000000001
Agent #9 | Ep #4 | Score: -965.12
Agent #9 | Ep #5 | Score: -954.16
Agent #9 | Ep #6 | Score: -972.1700000000001
Agent #9 | Ep #7 | Score: -967.15
Agent #9 | Ep #8 | Score: -978.1800000000001
Agent #9 | Ep #9 | Score: -980.1700000000001
Adding dffdec26-861c-4cb4-9444-a69599b68a42 to trainer learners
Adding 55dc8f0c-8a16-4293-8cdf-ce97651f4c21 to trainer learners
Adding afbc768a-d829-41e0-bed3-e2ef99dcb053 to trainer learners
Adding dbd1faf9-36bb-48d8-b59f-b67d6ff1d7c0 to trainer learners
Adding ea142b59-808e-4bb2-9712-23e423b21dd2 to trainer learners
Adding dc802921-dcd8-48f9-924e-5f623d4cdc1d to trainer learners
Adding 59fb3c5c-234b-4

Agent #5 | Ep #0 | Score: -85.14000000000064
Agent #5 | Ep #1 | Score: -84.11000000000064
Agent #5 | Ep #2 | Score: -83.05000000000061
Agent #5 | Ep #3 | Score: -84.20000000000067
Agent #5 | Ep #4 | Score: -83.98000000000062
Agent #5 | Ep #5 | Score: -84.94000000000065
Agent #5 | Ep #6 | Score: -85.11000000000065
Agent #5 | Ep #7 | Score: -83.08000000000061
Agent #5 | Ep #8 | Score: -84.07000000000059
Agent #5 | Ep #9 | Score: -84.14000000000064
Agent #6 | Ep #0 | Score: -956.14
Agent #6 | Ep #1 | Score: -980.16
Agent #6 | Ep #2 | Score: -960.14
Agent #6 | Ep #3 | Score: -969.11
Agent #6 | Ep #4 | Score: -964.0699999999999
Agent #6 | Ep #5 | Score: -966.0799999999999
Agent #6 | Ep #6 | Score: -969.12
Agent #6 | Ep #7 | Score: -962.15
Agent #6 | Ep #8 | Score: -959.13
Agent #6 | Ep #9 | Score: -980.16
Agent #7 | Ep #0 | Score: -986.03
Agent #7 | Ep #1 | Score: -979.03
Agent #7 | Ep #2 | Score: -976.03
Agent #7 | Ep #3 | Score: -974
Agent #7 | Ep #4 | Score: -989.02
Agent #7 | Ep #5 | Sc

...

Agent #5 | Ep #6 | Score: -979.09
Agent #5 | Ep #7 | Score: -965.0799999999999
Agent #5 | Ep #8 | Score: -982.05
Agent #5 | Ep #9 | Score: -990.09
Agent #6 | Ep #0 | Score: -82.0900000000006
Agent #6 | Ep #1 | Score: -84.02000000000059
Agent #6 | Ep #2 | Score: -84.25000000000068
Agent #6 | Ep #3 | Score: -84.22000000000068
Agent #6 | Ep #4 | Score: -84.2200000000007
Agent #6 | Ep #5 | Score: -83.0700000000006
Agent #6 | Ep #6 | Score: -84.04000000000059
Agent #6 | Ep #7 | Score: -84.03000000000061
Agent #6 | Ep #8 | Score: -82.06000000000058
Agent #6 | Ep #9 | Score: -85.05000000000065
Agent #7 | Ep #0 | Score: -80.99000000000055
Agent #7 | Ep #1 | Score: -84.0700000000006
Agent #7 | Ep #2 | Score: -85.22000000000068
Agent #7 | Ep #3 | Score: -83.0500000000006
Agent #7 | Ep #4 | Score: -83.19000000000067
Agent #7 | Ep #5 | Score: -84.14000000000064
Agent #7 | Ep #6 | Score: -83.17000000000063
Agent #7 | Ep #7 | Score: -80.99000000000055
Agent #7 | Ep #8 | Score: -85.08000000000068
Age

Agent #5 | Ep #0 | Score: -972.1800000000001
Agent #5 | Ep #1 | Score: -956.16
Agent #5 | Ep #2 | Score: -980.16
Agent #5 | Ep #3 | Score: -957.16
Agent #5 | Ep #4 | Score: -964.16
Agent #5 | Ep #5 | Score: -960.16
Agent #5 | Ep #6 | Score: -968.1700000000001
Agent #5 | Ep #7 | Score: -969.14
Agent #5 | Ep #8 | Score: -955.1700000000001
Agent #5 | Ep #9 | Score: -953.1700000000001
Agent #6 | Ep #0 | Score: -976.02
Agent #6 | Ep #1 | Score: -975.02
Agent #6 | Ep #2 | Score: -989.02
Agent #6 | Ep #3 | Score: -975.03
Agent #6 | Ep #4 | Score: -990.03
Agent #6 | Ep #5 | Score: -1000
Agent #6 | Ep #6 | Score: -976
Agent #6 | Ep #7 | Score: -993
Agent #6 | Ep #8 | Score: -984.01
Agent #6 | Ep #9 | Score: -979
Agent #7 | Ep #0 | Score: -86.1400000000007
Agent #7 | Ep #1 | Score: -84.16000000000065
Agent #7 | Ep #2 | Score: -83.03000000000058
Agent #7 | Ep #3 | Score: -85.07000000000065
Agent #7 | Ep #4 | Score: -83.08000000000061
Agent #7 | Ep #5 | Score: -84.08000000000061
Agent #7 | Ep #6 |

Agent #5 | Ep #0 | Score: -998
Agent #5 | Ep #1 | Score: -980
Agent #5 | Ep #2 | Score: -974.02
Agent #5 | Ep #3 | Score: -977
Agent #5 | Ep #4 | Score: -983.03
Agent #5 | Ep #5 | Score: -978.03
Agent #5 | Ep #6 | Score: -969.02
Agent #5 | Ep #7 | Score: -992
Agent #5 | Ep #8 | Score: -993
Agent #5 | Ep #9 | Score: -991.01
Agent #6 | Ep #0 | Score: -975.1800000000001
Agent #6 | Ep #1 | Score: -979.1700000000001
Agent #6 | Ep #2 | Score: -977.15
Agent #6 | Ep #3 | Score: -982.1700000000001
Agent #6 | Ep #4 | Score: -973.1700000000001
Agent #6 | Ep #5 | Score: -975.11
Agent #6 | Ep #6 | Score: -966.14
Agent #6 | Ep #7 | Score: -981.1700000000001
Agent #6 | Ep #8 | Score: -961.16
Agent #6 | Ep #9 | Score: -971.1700000000001
Agent #7 | Ep #0 | Score: -83.99000000000062
Agent #7 | Ep #1 | Score: -82.01000000000057
Agent #7 | Ep #2 | Score: -84.23000000000069
Agent #7 | Ep #3 | Score: -83.02000000000058
Agent #7 | Ep #4 | Score: -84.03000000000061
Agent #7 | Ep #5 | Score: -83.9900000000006


Agent #5 | Ep #0 | Score: -664.099999999998
Agent #5 | Ep #1 | Score: -676.1999999999979
Agent #5 | Ep #2 | Score: -678.1999999999979
Agent #5 | Ep #3 | Score: -667.079999999998
Agent #5 | Ep #4 | Score: -678.2199999999979
Agent #5 | Ep #5 | Score: -665.1499999999979
Agent #5 | Ep #6 | Score: -676.1899999999979
Agent #5 | Ep #7 | Score: -668.1499999999979
Agent #5 | Ep #8 | Score: -675.1699999999979
Agent #5 | Ep #9 | Score: -675.1799999999979
Agent #6 | Ep #0 | Score: -970.0799999999999
Agent #6 | Ep #1 | Score: -974.04
Agent #6 | Ep #2 | Score: -976.01
Agent #6 | Ep #3 | Score: -980.04
Agent #6 | Ep #4 | Score: -991.0699999999999
Agent #6 | Ep #5 | Score: -966.09
Agent #6 | Ep #6 | Score: -972.0799999999999
Agent #6 | Ep #7 | Score: -987.09
Agent #6 | Ep #8 | Score: -990.09
Agent #6 | Ep #9 | Score: -975.03
Agent #7 | Ep #0 | Score: -81.99000000000056
Agent #7 | Ep #1 | Score: -82.96000000000055
Agent #7 | Ep #2 | Score: -86.09000000000073
Agent #7 | Ep #3 | Score: -84.14000000000064

Agent #5 | Ep #0 | Score: -85.24000000000069
Agent #5 | Ep #1 | Score: -81.97000000000054
Agent #5 | Ep #2 | Score: -84.07000000000062
Agent #5 | Ep #3 | Score: -83.19000000000067
Agent #5 | Ep #4 | Score: -83.18000000000063
Agent #5 | Ep #5 | Score: -82.0900000000006
Agent #5 | Ep #6 | Score: -84.04000000000059
Agent #5 | Ep #7 | Score: -77.12000000000025
Agent #5 | Ep #8 | Score: -83.0700000000006
Agent #5 | Ep #9 | Score: -85.08000000000068
Agent #6 | Ep #0 | Score: -977.03
Agent #6 | Ep #1 | Score: -976
Agent #6 | Ep #2 | Score: -999
Agent #6 | Ep #3 | Score: -988
Agent #6 | Ep #4 | Score: -985.02
Agent #6 | Ep #5 | Score: -970
Agent #6 | Ep #6 | Score: -980.02
Agent #6 | Ep #7 | Score: -974.04
Agent #6 | Ep #8 | Score: -990.02
Agent #6 | Ep #9 | Score: -969.01
Agent #7 | Ep #0 | Score: -1000
Agent #7 | Ep #1 | Score: -969.01
Agent #7 | Ep #2 | Score: -990.01
Agent #7 | Ep #3 | Score: -994
Agent #7 | Ep #4 | Score: -968.0699999999999
Agent #7 | Ep #5 | Score: -993.02
Agent #7 | Ep 